In [ ]:
pip install tensorflow-determinism

In [ ]:
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
import rpy2.robjects.packages as rpackages
from rpy2.robjects import numpy2ri

numpy2ri.activate()

import logging
logging.getLogger('rpy2.rinterface_lib.callbacks').setLevel(logging.ERROR)

# ro.r['install.packages']('forecast')
utils = rpackages.importr('utils')
utils.install_packages('forecast')

forecast_pkg = importr('forecast')
stats = importr('stats')
generics = importr('generics')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.layers import LSTM, Dense, Activation, Dropout, Embedding
from tensorflow.keras.models import Sequential
# from tensorflow.keras import optimizers
import time
import tensorflow as tf

from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.graphics.tsaplots import plot_acf
# from pmdarima.arima import auto_arima

from tqdm import tqdm
import random
import os

In [ ]:
def set_seeds(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)

def set_global_determinism(seed):
    set_seeds(seed=seed)

    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

    tf.config.threading.set_inter_op_parallelism_threads(1)
    tf.config.threading.set_intra_op_parallelism_threads(1)

In [ ]:
def split_windows(x_lst, num_series, input_len, output_len):
  num_window_per_series = np.zeros(num_series, dtype=int)
  # Calculate the number of windows per series
  for i in range(num_series):
      num_window_per_series[i] = x_lst[i].shape[0] - input_len
  # Calculate the total number of windows
  num_window_total = np.sum(num_window_per_series)

  # Initialize the data frame
  x_data_frame = np.zeros((num_window_total, input_len + output_len))

  # Populate the data_frame
  win = 0
  for k in range(num_series):
      for win_k in range(num_window_per_series[k]):
          x_data_frame[win, :] = x_lst[k].loc[
              win_k:(win_k + input_len + output_len-1), 'data'
          ] # 0:30 include the last value
          win += 1

  # Initialize mean and standard deviation vectors
  mean_vec = np.zeros(num_window_total)
  std_vec = np.zeros(num_window_total)
  # Calculate mean and standard deviation for each window
  for win in range(num_window_total):
      mean_vec[win] = np.mean(x_data_frame[win, :input_len]) # :30, exclude the last one
      std_vec[win] = np.std(x_data_frame[win, :input_len])
  # Normalize the data frame
  norm_x_data_frame = np.zeros_like(x_data_frame)
  for win in range(num_window_total):
      norm_x_data_frame[win, :] = (
          x_data_frame[win, :] - mean_vec[win]
      ) / std_vec[win]

  return([x_data_frame, norm_x_data_frame, num_window_per_series, mean_vec, std_vec])

In [ ]:
def pred_and_results(x, model, x_data_frame, output_len, mean_vec_x, std_vec_x):
  x_predict = model.predict(x).reshape(-1)
  x_true = x_data_frame[:, -output_len]
  # Initialize an array for the original predictions
  orig_pred_x = np.zeros(len(x_predict))

  # Transform predictions back to the original scale
  for i in range(len(x_predict)):
      orig_pred_x[i] = x_predict[i] * std_vec_x[i] + mean_vec_x[i]

  return([orig_pred_x, x_true])

In [ ]:
def split_to_series(a, num_per_series, num_series):
  a_lst = []
  # Loop through each series
  for i in range(num_series):
      a_lst.append(a[:num_per_series[i]]) # exclude the last one
      a = a[num_per_series[i]:]  # Remove used elements from a
  return(a_lst)

In [ ]:
def metrics(num_series, true_demand_lst, pred_test_demand_lst):
  # Initialize arrays to store metrics
  rmse_vec = np.zeros(num_series)
  mae_vec = np.zeros(num_series)
  smape_vec = np.zeros(num_series)

  # Compute metrics for each series
  for i in range(num_series):
      true_values = true_demand_lst[i]
      predicted_values = pred_test_demand_lst[i]

      # RMSE
      rmse_vec[i] = np.sqrt(np.mean((predicted_values - true_values) ** 2))

      # MAE
      mae_vec[i] = np.mean(np.abs(predicted_values - true_values))

      # sMAPE
      smape_vec[i] = 2 * np.mean(
          np.abs(predicted_values - true_values) /
          (np.abs(predicted_values) + np.abs(true_values))
      )

  # Aggregate results
  mean_rmse = np.mean(rmse_vec)
  median_rmse = np.median(rmse_vec)

  mean_mae = np.mean(mae_vec)
  median_mae = np.median(mae_vec)

  mean_smape = np.mean(smape_vec)
  median_smape = np.median(smape_vec)

  # Print results
  print("Mean RMSE:", mean_rmse)
  print("Median RMSE:", median_rmse)

  print("Mean MAE:", mean_mae)
  print("Median MAE:", median_mae)

  print("Mean sMAPE:", mean_smape)
  print("Median sMAPE:", median_smape)

  # cum_rmse
  cum_rmse_lst = [
    np.mean([
        np.sqrt(
            np.mean((np.array(true_demand_lst[i][:t+1]) - np.array(pred_test_demand_lst[i][:t+1]))**2)
        )
        for t in range(len(true_demand_lst[i]))
    ])
    for i in range(len(true_demand_lst))
  ]

  cum_mae_lst = [
    np.mean([
        np.mean(
            np.abs(np.array(true_demand_lst[i][:t+1]) - np.array(pred_test_demand_lst[i][:t+1]))
            )
        for t in range(len(true_demand_lst[i]))
    ])
    for i in range(len(true_demand_lst))
  ]

  cum_smape_lst = [
    np.mean([
        2* np.mean(
            np.abs(np.array(true_demand_lst[i][:t+1]) - np.array(pred_test_demand_lst[i][:t+1])) /
            (np.abs(np.array(true_demand_lst[i][:t+1])) + np.abs(np.array(pred_test_demand_lst[i][:t+1])))
            )
        for t in range(len(true_demand_lst[i]))
    ])
    for i in range(len(true_demand_lst))
  ]

  mean_cum_rmse = np.mean(cum_rmse_lst)
  median_cum_rmse = np.median(cum_rmse_lst)
  mean_cum_mae = np.mean(cum_mae_lst)
  median_cum_mae = np.median(cum_mae_lst)
  mean_cum_smape = np.mean(cum_smape_lst)
  median_cum_smape = np.median(cum_smape_lst)

  # print("Mean Cum RMSE:", mean_cum_rmse)
  # print("Median Cum RMSE:", median_cum_rmse)

  return([mean_rmse, median_rmse, mean_mae, median_mae, mean_smape, median_smape,
          mean_cum_rmse, median_cum_rmse, mean_cum_mae, median_cum_mae, mean_cum_smape, median_cum_smape])

In [ ]:
def build_mlp_model(input_len, output_len):
  model = Sequential()
  model.add(Dense(units=8, activation='tanh', input_dim= input_len))
  model.add(Dense(4))
  model.add(Dense(output_len))
  model.compile(loss = 'mean_squared_error', optimizer = 'adam')
  print(model.summary())
  return(model)

In [ ]:
def build_lstm_model(input_len):
  model = Sequential()

  model.add(LSTM(units = 16, return_sequences = False, input_shape=(input_len, 1)))
  model.add(Dropout(0.5))

  # model.add(LSTM(8, return_sequences = False))
  # model.add(Dropout(0.2))

  # model.add(Dense(64))
  # model.add(Dense(units=8, activation='tanh', input_dim= input_len))
  model.add(Dense(4))
  model.add(Dense(1))

  # model.add(Activation('relu'))

  start = time.time()

  model.compile(loss = 'mean_squared_error', optimizer = 'adam')
  print(model.summary())
  return(model)

In [ ]:
def run_main(data, info, input_len, output_len = 1, seasonal_period = 12,
             build_model = build_mlp_model, batch_size = 64, epochs = 30, random_seed = 42):
  num_series = info.shape[0]
  print('num_series:', num_series)

  data_lst = []
  for i in range(num_series):
    data_lst.append(data[data['idx']==(i+1)])

  xtest_idx = np.array(info['test_idx'])
  xtest_lst, xtrain_lst = [], []
  for i in range(num_series):
    xtrain_lst.append(data_lst[i][data_lst[i]['time']<xtest_idx[i]].reset_index(drop=True))
    xtest_lst.append(data_lst[i][data_lst[i]['time']>=xtest_idx[i] - input_len].reset_index(drop=True))

  train_data_frame, norm_train_data_frame, num_window_per_series_train, mean_vec_train, std_vec_train = \
  split_windows(xtrain_lst, num_series, input_len, output_len)
  # Calculate the total number of windows
  num_window_total_train = np.sum(num_window_per_series_train)
  print('total number of windows:', num_window_total_train)

  # [batch, timesteps, feature]
  train_X = norm_train_data_frame[:,:-output_len].reshape((-1,input_len,1))
  train_Y = norm_train_data_frame[:,-output_len].reshape((-1,output_len,1))

  print('start training')
  tf.keras.backend.clear_session()
  # Call the above function with seed value
  set_global_determinism(seed=random_seed)
  model = build_model(input_len, output_len)
  history = model.fit(train_X, train_Y, batch_size = batch_size, epochs = epochs, verbose=1, shuffle=False)

  orig_pred_train, train_true = \
  pred_and_results(train_X, model, train_data_frame, output_len, mean_vec_train, std_vec_train)
  train_rmse_without_adj = np.sqrt(np.mean((orig_pred_train - train_true)**2))
  print('train_rmse_without_adj:', train_rmse_without_adj)

  # adj models
  diff = orig_pred_train - train_true
  diff_lst = split_to_series(diff, num_window_per_series_train, num_series)
  pred_lst = split_to_series(orig_pred_train, num_window_per_series_train, num_series)

  # Initialize p-value vector
  p_vec = np.zeros(num_series)
  # Perform the Ljung-Box test for each series
  for i in range(num_series):
      lb_test = acorr_ljungbox(diff_lst[i], lags=int(np.log(len(diff_lst[i]))), return_df=True)
      p_vec[i] = lb_test['lb_pvalue'].iloc[-1]  # Get the p-value from the last lag
  # Count and get indices of series with p-values < 0.05
  hetero_series_idx = np.where(p_vec < 0.05)[0]
  print(f"Number of heteroscedastic series: {len(hetero_series_idx)}")

  # Initialize list for adjusted models
  adj_models = [0] * num_series

  # Adjust predictions for heteroscedastic series
  for i in tqdm(hetero_series_idx):
      # Fit ARIMA model
      ts_data = ro.r['ts'](ro.FloatVector(diff_lst[i]), frequency=seasonal_period)
      arima_model = forecast_pkg.auto_arima(ts_data)
      adj_models[i] = arima_model
      # Update predictions by subtracting fitted values from the ARIMA model
      adj_values = np.array(ro.r['forecast'](arima_model).rx2('fitted'))
      pred_lst[i] = pred_lst[i] - adj_values

  # Combine predictions back into a single array
  adj_forecast = np.concatenate(pred_lst)
  # Compute RMSE
  train_rmse_with_adj = np.sqrt(np.mean((adj_forecast - train_true) ** 2))
  print("train_rmse_with_adj:", train_rmse_with_adj)

  print('start testing')
  test_data_frame, norm_test_data_frame, num_window_per_series_test, mean_vec_test, std_vec_test = \
  split_windows(xtest_lst, num_series, input_len, output_len)
  true_demand_lst = split_to_series(test_data_frame[:,-1].reshape(-1), num_window_per_series_test, num_series)
  test_predict = model.predict(norm_test_data_frame[:, :input_len].reshape((-1,input_len,1)))
  orig_test_predict = test_predict.reshape(-1) * std_vec_test + mean_vec_test
  pred_test_demand_lst = split_to_series(orig_test_predict, num_window_per_series_test, num_series)
  print('metrics before adjusting')
  res_without_adj = metrics(num_series, true_demand_lst, pred_test_demand_lst)

  # Adjusted forecast
  for i in hetero_series_idx:
    adj = np.array(generics.forecast(adj_models[i], h = num_window_per_series_test[i]).rx2('mean'))
    pred_test_demand_lst[i] = pred_test_demand_lst[i] - adj
  print('metrics after adjusting')
  res_with_adj = metrics(num_series, true_demand_lst, pred_test_demand_lst)

  metrics_res = {'res_without_adj': res_without_adj,
               'res_with_adj': res_with_adj}
  df = pd.DataFrame(metrics_res)
  return(df)

In [ ]:
tourism_data = pd.read_csv('tourism_data.csv')
tourism_info = pd.read_csv('tourism_info.csv')

# tourism_data = pd.read_csv('m3_5_data.csv')
# tourism_info = pd.read_csv('m3_5_info.csv')

In [ ]:
# tourism mlp, final
run_main(tourism_data, tourism_info, input_len = 24, output_len = 1, seasonal_period = 12,
         build_model = build_mlp_model, batch_size = 32, epochs = 30, random_seed = 3407)

num_series: 366
start training


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 8)                   │             200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4)                   │              36 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │               5 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 241 (964.00 B)

...

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/30
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 1.2834
Epoch 2/30
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7788
Epoch 3/30
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.7673
Epoch 4/30
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.7607
Epoch 5/30
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 0.7566
Epoch 6/30
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.7543
Epoch 7/30
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.7532
Epoch 8/30
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.7524
Epoch 9/30
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.7517
Epoch 10/30
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 0.7511
Epoch 11/30
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.7504
Epoch 12/30
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.7499
Epoch 13/30
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.7493
Epoch 14/30
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.7489
Epoch 15/30
2866/2866 ━━

100%|██████████| 186/186 [02:50<00:00,  1.09it/s]


train_rmse_with_adj: 4098.33927406212
start testing
275/275 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
metrics before adjusting
Mean RMSE: 1876.4973677570817
Median RMSE: 487.1265648208632
Mean MAE: 1446.438951217619
Median MAE: 380.08816327175543
Mean sMAPE: 0.16678930210965015
Median sMAPE: 0.13916624208307699
Mean Cum RMSE: 1913.4579063771191
Median Cum RMSE: 511.0505191387956
metrics after adjusting
Mean RMSE: 1872.2358240773606
Median RMSE: 487.1265648208632
Mean MAE: 1435.7661329476514
Median MAE: 377.9997686502156
Mean sMAPE: 0.16673351861893151
Median sMAPE: 0.13897484901254864
Mean Cum RMSE: 1898.2188174053142
Median Cum RMSE: 503.73783106227273


,res_without_adj,res_with_adj
0,1876.497368,1872.235824
1,487.126565,487.126565
2,1446.438951,1435.766133
3,380.088163,377.999769
4,0.166789,0.166734
5,0.139166,0.138975
6,1913.457906,1898.218817
7,511.050519,503.737831
8,1539.330574,1529.742922
9,393.885128,386.049999


In [ ]:
# tourism lstm, final
run_main(tourism_data, tourism_info, input_len = 24, output_len = 1, seasonal_period = 12,
         build_model = build_lstm_model, batch_size = 32, epochs = 30, random_seed = 42)

num_series: 366
total number of windows: 91712
start training


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 24, 16)              │           1,152 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 24, 16)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 8)                   │             800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 8)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 4)                   │              36 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │               5 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,993 (7.79 KB)

...

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/30
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 44s 14ms/step - loss: 1.3898
Epoch 2/30
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 82s 14ms/step - loss: 1.0367
Epoch 3/30
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 81s 14ms/step - loss: 0.9727
Epoch 4/30
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 42s 15ms/step - loss: 0.9439
Epoch 5/30
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 81s 14ms/step - loss: 0.9123
Epoch 6/30
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 41s 14ms/step - loss: 0.8717
Epoch 7/30
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 41s 14ms/step - loss: 0.8379
Epoch 8/30
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 82s 14ms/step - loss: 0.8180
Epoch 9/30
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 41s 14ms/step - loss: 0.8133
Epoch 10/30
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 41s 14ms/step - loss: 0.8052
Epoch 11/30
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 82s 14ms/step - loss: 0.8037
Epoch 12/30
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 40s 14ms/step - loss: 0.8005
Epoch 13/30
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 41s 14ms/step - loss: 0.7925
Epoch 14/30
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 41s 14ms/step - loss: 0.7

100%|██████████| 198/198 [02:27<00:00,  1.35it/s]


train_rmse_with_adj: 3957.7249646182763
start testing
275/275 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step
metrics before adjusting
Mean RMSE: 1979.0545458816514
Median RMSE: 465.06579857465084
Mean MAE: 1508.3336594381562
Median MAE: 369.2224580914519
Mean sMAPE: 0.16495698031817274
Median sMAPE: 0.14192862432530337
metrics after adjusting
Mean RMSE: 1888.8514297026773
Median RMSE: 460.24035663493316
Mean MAE: 1464.6169824743013
Median MAE: 363.9702679372957
Mean sMAPE: 0.16434749117576875
Median sMAPE: 0.14182135653311165


,res_without_adj,res_with_adj
0,1979.054546,1888.851430
1,465.065799,460.240357
2,1508.333659,1464.616982
3,369.222458,363.970268
4,0.164957,0.164347
5,0.141929,0.141821
6,1963.280541,1903.357393
7,495.335867,483.842696
8,1540.164182,1536.206038
9,384.459229,387.552398


In [ ]:
hospital_data = pd.read_csv('hospital_data.csv')
hospital_info = pd.read_csv('hospital_info.csv')

In [ ]:
# hospital mlp
def build_mlp_model(input_len):
  model = Sequential()
  model.add(Dense(units=16, activation='tanh', input_dim= input_len))
  # model.add(Dense(4))
  model.add(Dense(1))
  model.compile(loss = 'mean_squared_error', optimizer = 'adam')
  print(model.summary())
  return(model)

hospital_mlp_df = run_main(hospital_data, hospital_info, input_len = 24, output_len = 1, seasonal_period = 12,
                       build_model = build_mlp_model, batch_size = 32, epochs = 30, random_seed = 42)
hospital_mlp_df

num_series: 767
total number of windows: 36816
start training


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 16)                  │             400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 417 (1.63 KB)

...

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/30
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1.5297
Epoch 2/30
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1.2757
Epoch 3/30
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1.2656
Epoch 4/30
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1.2607
Epoch 5/30
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1.2580
Epoch 6/30
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 1.2560
Epoch 7/30
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1.2543
Epoch 8/30
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1.2528
Epoch 9/30
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1.2513
Epoch 10/30
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 1.2499
Epoch 11/30
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 1.2484
Epoch 12/30
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 1.2469
Epoch 13/30
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 1.2455
Epoch 14/30
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1.2440
Epoch 15/30
1151/1151 ━━

100%|██████████| 92/92 [01:20<00:00,  1.14it/s]


train_rmse_with_adj: 51.581079962166754
start testing
288/288 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
metrics before adjusting
Mean RMSE: 20.709553931619308
Median RMSE: 7.838619583352475
Mean MAE: 16.995502155453647
Median MAE: 6.3474110768574805
Mean sMAPE: 0.16393688745628798
Median sMAPE: 0.15527867704619538
Mean Cum RMSE: 20.67362537761336
Median Cum RMSE: 7.83253644753479
metrics after adjusting
Mean RMSE: 20.672786604255368
Median RMSE: 7.838619583352475
Mean MAE: 16.966428680257202
Median MAE: 6.365597596905213
Mean sMAPE: 0.16480295426060326
Median sMAPE: 0.15545874540588558
Mean Cum RMSE: 20.59606310620804
Median Cum RMSE: 7.824715432724978


,res_without_adj,res_with_adj
0,20.709554,20.672787
1,7.838620,7.838620
2,16.995502,16.966429
3,6.347411,6.365598
4,0.163937,0.164803
5,0.155279,0.155459
6,20.673625,20.596063
7,7.832536,7.824715
8,17.650893,17.573983
9,6.600078,6.615787


In [ ]:
# hospital lstm
def build_lstm_model(input_len, output_len):
  model = Sequential()
  model.add(LSTM(units = 8, return_sequences = False, input_shape=(input_len, 1)))
  model.add(Dropout(0.5))
  model.add(Dense(4))
  model.add(Dense(output_len))
  model.compile(loss = 'mean_squared_error', optimizer = 'adam')
  print(model.summary())
  return(model)

hospital_lstm_df = run_main(hospital_data, hospital_info, input_len = 24, output_len = 1, seasonal_period = 12,
                            build_model = build_lstm_model, batch_size = 32, epochs = 30, random_seed = 3407)
hospital_lstm_df

num_series: 767
total number of windows: 36816
start training


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 8)                   │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 8)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 4)                   │              36 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │               5 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 361 (1.41 KB)

...

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/30
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - loss: 1.3903
Epoch 2/30
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - loss: 1.3346
Epoch 3/30
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 19s 9ms/step - loss: 1.3292
Epoch 4/30
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 21s 10ms/step - loss: 1.3267
Epoch 5/30
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - loss: 1.3228
Epoch 6/30
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 19s 9ms/step - loss: 1.3235
Epoch 7/30
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - loss: 1.3258
Epoch 8/30
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - loss: 1.3209
Epoch 9/30
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 19s 9ms/step - loss: 1.3183
Epoch 10/30
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 22s 10ms/step - loss: 1.3210
Epoch 11/30
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 20s 10ms/step - loss: 1.3165
Epoch 12/30
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 20s 10ms/step - loss: 1.3196
Epoch 13/30
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - loss: 1.3125
Epoch 14/30
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - loss: 1.3146

100%|██████████| 91/91 [01:21<00:00,  1.12it/s]


train_rmse_with_adj: 55.98583904205714
start testing
288/288 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
metrics before adjusting
Mean RMSE: 22.32505378501847
Median RMSE: 7.76999360340417
Mean MAE: 18.31962348106394
Median MAE: 6.346959953028677
Mean sMAPE: 0.16455499091092168
Median sMAPE: 0.15527681381560796
metrics after adjusting
Mean RMSE: 21.921784412068135
Median RMSE: 7.829632811321031
Mean MAE: 17.971330139050558
Median MAE: 6.369045574899192
Mean sMAPE: 0.16509012473968446
Median sMAPE: 0.15482164811888788


,res_without_adj,res_with_adj
0,22.325054,21.921784
1,7.769994,7.829633
2,18.319623,17.971330
3,6.346960,6.369046
4,0.164555,0.165090
5,0.155277,0.154822
6,22.055531,21.887983
7,7.724806,7.724806
8,18.739189,18.653899
9,6.557925,6.546463


In [ ]:
cif_data = pd.read_csv('cif2016_data.csv')
cif_info = pd.read_csv('cif2016_info.csv')

In [ ]:
# cif mlp
def build_mlp_model(input_len):
  model = Sequential()
  model.add(Dense(units=8, activation='tanh', input_dim= input_len))
  # model.add(Dense(4))
  model.add(Dense(1))
  model.compile(loss = 'mean_squared_error', optimizer = 'adam')
  print(model.summary())
  return(model)

cif_mlp_df = run_main(cif_data, cif_info, input_len = 12, output_len = 1, seasonal_period = 12,
                       build_model = build_mlp_model, batch_size = 32, epochs = 30, random_seed = 3407)
cif_mlp_df

num_series: 72
total number of windows: 5470
start training


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 8)                   │             104 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │               9 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 113 (452.00 B)

...

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.8167
Epoch 2/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.4144
Epoch 3/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.3500
Epoch 4/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.3326
Epoch 5/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.3240
Epoch 6/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.3181
Epoch 7/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.3134
Epoch 8/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.3094
Epoch 9/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.3060
Epoch 10/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.3029
Epoch 11/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.3001
Epoch 12/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.2975
Epoch 13/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.2950
Epoch 14/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.2926
Epoch 15/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

100%|██████████| 34/34 [04:25<00:00,  7.81s/it]


train_rmse_with_adj: 4385733.883959476
start testing
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
metrics before adjusting
Mean RMSE: 352476.76701412373
Median RMSE: 86.94185552018125
Mean MAE: 278145.70891703025
Median MAE: 70.77841428913547
Mean sMAPE: 0.09332545420272903
Median sMAPE: 0.07488956098436537
Mean Cum RMSE: 293090.6375080792
Median Cum RMSE: 78.25860214832153
metrics after adjusting
Mean RMSE: 356009.9759363703
Median RMSE: 62.5263415804537
Mean MAE: 281323.70340137143
Median MAE: 45.94219823502527
Mean sMAPE: 0.08028713457317439
Median sMAPE: 0.061972973403787185
Mean Cum RMSE: 308556.69615761243
Median Cum RMSE: 52.03617106489243


,res_without_adj,res_with_adj
0,352476.767014,356009.975936
1,86.941856,62.526342
2,278145.708917,281323.703401
3,70.778414,45.942198
4,0.093325,0.080287
5,0.074890,0.061973
6,293090.637508,308556.696158
7,78.258602,52.036171
8,239905.587200,256551.538327
9,67.904876,44.981757


In [ ]:
# cif lstm
def build_lstm_model(input_len, output_len):
  model = Sequential()
  model.add(LSTM(units = 8, return_sequences = False, input_shape=(input_len, 1)))
  model.add(Dropout(0.5))
  model.add(Dense(4))
  model.add(Dense(output_len))
  model.compile(loss = 'mean_squared_error', optimizer = 'adam')
  print(model.summary())
  return(model)

cif_lstm_df = run_main(cif_data, cif_info, input_len = 12, output_len = 1, seasonal_period = 12,
                       build_model = build_lstm_model, batch_size = 32, epochs = 30, random_seed = 3407)
cif_lstm_df

num_series: 72
total number of windows: 5470
start training


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 8)                   │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 8)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 4)                   │              36 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │               5 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 361 (1.41 KB)

...

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 1.5892
Epoch 2/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 1.4659
Epoch 3/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 1.4610
Epoch 4/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 1.4655
Epoch 5/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 1.4471
Epoch 6/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 1.4408
Epoch 7/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 1.4737
Epoch 8/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 1.4632
Epoch 9/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 1.4355
Epoch 10/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 1.4429
Epoch 11/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 1.4408
Epoch 12/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 1.4399
Epoch 13/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 1.4370
Epoch 14/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 1.4269
Epoch 15/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/ste

100%|██████████| 37/37 [02:54<00:00,  4.72s/it]


train_rmse_with_adj: 4518792.666329101
start testing
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
metrics before adjusting
Mean RMSE: 338097.60586401797
Median RMSE: 95.01669923579757
Mean MAE: 261428.802898736
Median MAE: 78.69440891382553
Mean sMAPE: 0.1009581276449551
Median sMAPE: 0.08100419198747366
metrics after adjusting
Mean RMSE: 295955.11459323484
Median RMSE: 62.16030775367176
Mean MAE: 218278.24365485122
Median MAE: 53.10040584310728
Mean sMAPE: 0.08203484925837193
Median sMAPE: 0.0589298399713311


,res_without_adj,res_with_adj
0,338097.605864,295955.114593
1,95.016699,62.160308
2,261428.802899,218278.243655
3,78.694409,53.100406
4,0.100958,0.082035
5,0.081004,0.058930
6,268568.973354,266999.873611
7,87.127801,54.156285
8,215052.708069,207039.528117
9,72.228685,50.356477


In [ ]:
import copy

In [ ]:
def run_main_m3(data, info, input_len, output_len = 1, seasonal_period = 12,
                build_model = build_mlp_model, batch_size = 64, epochs = 30, random_seed = 42):
  num_series = info.shape[0]
  print('num_series:', num_series)

  data_lst = []
  for i in range(num_series):
    data_lst.append(data[data['idx']==(i+1)])

  xtest_idx = np.array(info['test_idx'])
  xtest_lst, xtrain_lst = [], []
  for i in range(num_series):
    xtrain_lst.append(data_lst[i][data_lst[i]['time']<xtest_idx[i]].reset_index(drop=True))
    xtest_lst.append(data_lst[i][data_lst[i]['time']>=xtest_idx[i] - input_len].reset_index(drop=True))

  train_data_frame, norm_train_data_frame, num_window_per_series_train, mean_vec_train, std_vec_train = \
  split_windows(xtrain_lst, num_series, input_len, output_len)
  # Calculate the total number of windows
  num_window_total_train = np.sum(num_window_per_series_train)
  print('total number of windows:', num_window_total_train)

  # [batch, timesteps, feature]
  train_X = norm_train_data_frame[:,:-output_len].reshape((-1,input_len,1))
  train_Y = norm_train_data_frame[:,-output_len].reshape((-1,output_len,1))

  print('start training')
  tf.keras.backend.clear_session()
  # Call the above function with seed value
  set_global_determinism(seed=random_seed)
  model = build_model(input_len, output_len)
  history = model.fit(train_X, train_Y, batch_size = batch_size, epochs = epochs, verbose=1, shuffle=False)

  orig_pred_train, train_true = \
  pred_and_results(train_X, model, train_data_frame, output_len, mean_vec_train, std_vec_train)
  train_rmse_without_adj = np.sqrt(np.mean((orig_pred_train - train_true)**2))
  print('train_rmse_without_adj:', train_rmse_without_adj)

  # adj models
  diff = orig_pred_train - train_true
  diff_lst = split_to_series(diff, num_window_per_series_train, num_series)
  pred_lst = split_to_series(orig_pred_train, num_window_per_series_train, num_series)

  # Initialize p-value vector
  p_vec = np.zeros(num_series)
  # Perform the Ljung-Box test for each series
  for i in range(num_series):
      lb_test = acorr_ljungbox(diff_lst[i], lags=int(np.log(len(diff_lst[i]))), return_df=True)
      p_vec[i] = lb_test['lb_pvalue'].iloc[-1]  # Get the p-value from the last lag
  # Count and get indices of series with p-values < 0.05
  hetero_series_idx = np.where(p_vec < 0.05)[0]
  print(f"Number of heteroscedastic series: {len(hetero_series_idx)}")

  # Initialize list for adjusted models
  adj_models = [0] * num_series

  # Adjust predictions for heteroscedastic series
  for i in tqdm(hetero_series_idx):
      # Fit ARIMA model
      ts_data = ro.r['ts'](ro.FloatVector(diff_lst[i]), frequency=seasonal_period)
      arima_model = forecast_pkg.auto_arima(ts_data)
      adj_models[i] = arima_model
      # Update predictions by subtracting fitted values from the ARIMA model
      adj_values = np.array(ro.r['forecast'](arima_model).rx2('fitted'))
      pred_lst[i] = pred_lst[i] - adj_values

  # Combine predictions back into a single array
  adj_forecast = np.concatenate(pred_lst)
  # Compute RMSE
  train_rmse_with_adj = np.sqrt(np.mean((adj_forecast - train_true) ** 2))
  print("train_rmse_with_adj:", train_rmse_with_adj)

  print('start testing')
  test_data_frame, norm_test_data_frame, num_window_per_series_test, mean_vec_test, std_vec_test = \
  split_windows(xtest_lst, num_series, input_len, output_len)
  true_demand_lst = split_to_series(test_data_frame[:,-1].reshape(-1), num_window_per_series_test, num_series)
  test_predict = model.predict(norm_test_data_frame[:, :input_len].reshape((-1,input_len,1)))
  orig_test_predict = test_predict.reshape(-1) * std_vec_test + mean_vec_test
  pred_test_demand_lst = split_to_series(orig_test_predict, num_window_per_series_test, num_series)
  print('metrics before adjusting')
  # res_without_adj = metrics(num_series, true_demand_lst, pred_test_demand_lst)
  pred_test_demand_lst_without_adj = copy.deepcopy(pred_test_demand_lst)

  # Adjusted forecast
  for i in hetero_series_idx:
    adj = np.array(generics.forecast(adj_models[i], h = num_window_per_series_test[i]).rx2('mean'))
    pred_test_demand_lst[i] = pred_test_demand_lst[i] - adj
  print('metrics after adjusting')
  # res_with_adj = metrics(num_series, true_demand_lst, pred_test_demand_lst)
  pred_test_demand_lst_with_adj = pred_test_demand_lst

  # metrics_res = {'res_without_adj': res_without_adj,
  #              'res_with_adj': res_with_adj}
  # df = pd.DataFrame(metrics_res)
  return([num_series, true_demand_lst, pred_test_demand_lst_without_adj, pred_test_demand_lst_with_adj])

In [ ]:
m3_1_data = pd.read_csv('m3_1_data.csv')
m3_1_info = pd.read_csv('m3_1_info.csv')
m3_2_data = pd.read_csv('m3_2_data.csv')
m3_2_info = pd.read_csv('m3_2_info.csv')
m3_3_data = pd.read_csv('m3_3_data.csv')
m3_3_info = pd.read_csv('m3_3_info.csv')
m3_4_data = pd.read_csv('m3_4_data.csv')
m3_4_info = pd.read_csv('m3_4_info.csv')
m3_5_data = pd.read_csv('m3_5_data.csv')
m3_5_info = pd.read_csv('m3_5_info.csv')

In [ ]:
def build_mlp_model(input_len):
  model = Sequential()
  model.add(Dense(units=16, activation='tanh', input_dim= input_len))
  # model.add(Dense(4))
  model.add(Dense(1))
  model.compile(loss = 'mean_squared_error', optimizer = 'adam')
  print(model.summary())
  return(model)

In [ ]:
num_series_m3_1, true_demand_lst_m3_1, pred_test_demand_lst_without_adj_m3_1, pred_test_demand_lst_with_adj_m3_1 = \
run_main_m3(m3_1_data, m3_1_info, input_len = 24, output_len = 1, seasonal_period = 12,
            build_model = build_mlp_model, batch_size = 32, epochs = 30, random_seed = 3407)

num_series: 111
total number of windows: 9028
start training


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 16)                  │             400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 417 (1.63 KB)

 Trainable params: 417 (1.63 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/30
283/283 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.6126
Epoch 2/30
283/283 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4622
Epoch 3/30
283/283 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3642
Epoch 4/30
283/283 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3325
Epoch 5/30
283/283 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3180
Epoch 6/30
283/283 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3098
Epoch 7/30
283/283 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3044
Epoch 8/30
283/283 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3003
Epoch 9/30
283/283 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2970
Epoch 10/30
283/283 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2942
Epoch 11/30
283/283 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2917
Epoch 12/30
283/283 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2895
Epoch 13/30
283/283 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2875
Epoch 14/30
283/283 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2856
Epoch 15/30
283/283 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

100%|██████████| 70/70 [04:36<00:00,  3.95s/it]


train_rmse_with_adj: 787.3945168207654
start testing
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
metrics before adjusting
metrics after adjusting


In [ ]:
def metrics_m3(num_series, true_demand_lst, pred_test_demand_lst_without_adj, pred_test_demand_lst_with_adj):
  res_without_adj = metrics(num_series, true_demand_lst, pred_test_demand_lst_without_adj)
  res_with_adj = metrics(num_series, true_demand_lst, pred_test_demand_lst_with_adj)
  metrics_res = {'res_without_adj': res_without_adj,
               'res_with_adj': res_with_adj}
  df = pd.DataFrame(metrics_res)
  return(df)

In [ ]:
metrics_m3(num_series_m3_1, true_demand_lst_m3_1,
           pred_test_demand_lst_without_adj_m3_1, pred_test_demand_lst_with_adj_m3_1)

Mean RMSE: 243.2973707656905
Median RMSE: 47.580274967348515
Mean MAE: 187.93869330780515
Median MAE: 36.58120734700373
Mean sMAPE: 0.04085124084634798
Median sMAPE: 0.005637307180995616
Mean RMSE: 211.45494305423568
Median RMSE: 44.26609036761613
Mean MAE: 170.36266597966667
Median MAE: 36.566302375630386
Mean sMAPE: 0.04078688189126541
Median sMAPE: 0.005637307180995616


,res_without_adj,res_with_adj
0,243.297371,211.454943
1,47.580275,44.266090
2,187.938693,170.362666
3,36.581207,36.566302
4,0.040851,0.040787
5,0.005637,0.005637
6,252.347411,211.383901
7,39.576784,38.590161
8,202.004401,178.134200
9,33.364113,31.590819


In [ ]:
num_series_m3_2, true_demand_lst_m3_2, pred_test_demand_lst_without_adj_m3_2, pred_test_demand_lst_with_adj_m3_2 = \
run_main_m3(m3_2_data, m3_2_info, input_len = 24, output_len = 1, seasonal_period = 12,
            build_model = build_mlp_model, batch_size = 32, epochs = 30, random_seed = 3407)

num_series_m3_3, true_demand_lst_m3_3, pred_test_demand_lst_without_adj_m3_3, pred_test_demand_lst_with_adj_m3_3 = \
run_main_m3(m3_3_data, m3_3_info, input_len = 24, output_len = 1, seasonal_period = 12,
            build_model = build_mlp_model, batch_size = 32, epochs = 30, random_seed = 3407)

num_series_m3_4, true_demand_lst_m3_4, pred_test_demand_lst_without_adj_m3_4, pred_test_demand_lst_with_adj_m3_4 = \
run_main_m3(m3_4_data, m3_4_info, input_len = 24, output_len = 1, seasonal_period = 12,
            build_model = build_mlp_model, batch_size = 32, epochs = 30, random_seed = 3407)

num_series_m3_5, true_demand_lst_m3_5, pred_test_demand_lst_without_adj_m3_5, pred_test_demand_lst_with_adj_m3_5 = \
run_main_m3(m3_5_data, m3_5_info, input_len = 24, output_len = 1, seasonal_period = 12,
            build_model = build_mlp_model, batch_size = 32, epochs = 30, random_seed = 3407)

num_series: 145
total number of windows: 11948
start training


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 16)                  │             400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 417 (1.63 KB)

...

 Trainable params: 417 (1.63 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/30
751/751 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 1.1916
Epoch 2/30
751/751 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.0190
Epoch 3/30
751/751 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.0068
Epoch 4/30
751/751 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.9992
Epoch 5/30
751/751 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.9938
Epoch 6/30
751/751 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.9897
Epoch 7/30
751/751 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.9864
Epoch 8/30
751/751 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.9835
Epoch 9/30
751/751 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.9809
Epoch 10/30
751/751 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.9784
Epoch 11/30
751/751 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.9755
Epoch 12/30
751/751 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.9725
Epoch 13/30
751/751 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.9697
Epoch 14/30
751/751 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.9675
Epoch 15/30
751/751 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step

100%|██████████| 56/56 [02:01<00:00,  2.16s/it]


train_rmse_with_adj: 975.8371864057875
start testing
267/267 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
metrics before adjusting
metrics after adjusting


In [ ]:
metrics_m3(num_series_m3_2, true_demand_lst_m3_2,
           pred_test_demand_lst_without_adj_m3_2, pred_test_demand_lst_with_adj_m3_2)

Mean RMSE: 558.5163649909466
Median RMSE: 198.75076738228444
Mean MAE: 441.29904765551277
Median MAE: 159.91917707743974
Mean sMAPE: 0.07090061256252568
Median sMAPE: 0.021753540533553103
Mean RMSE: 546.2804141308742
Median RMSE: 198.75076738228444
Mean MAE: 427.86703585605983
Median MAE: 156.04115071307913
Mean sMAPE: 0.06839041328470022
Median sMAPE: 0.022248855518829033


,res_without_adj,res_with_adj
0,558.516365,546.280414
1,198.750767,198.750767
2,441.299048,427.867036
3,159.919177,156.041151
4,0.070901,0.068390
5,0.021754,0.022249
6,574.034492,558.436341
7,197.641609,207.601840
8,473.542851,453.393343
9,172.672273,172.741107


In [ ]:
metrics_m3(num_series_m3_3, true_demand_lst_m3_3,
           pred_test_demand_lst_without_adj_m3_3, pred_test_demand_lst_with_adj_m3_3)

Mean RMSE: 540.6036384388124
Median RMSE: 334.9704845103582
Mean MAE: 421.78781821644736
Median MAE: 263.022019706578
Mean sMAPE: 0.08350634582317838
Median sMAPE: 0.05443975721535998
Mean RMSE: 541.2451028681145
Median RMSE: 335.6550234972167
Mean MAE: 422.08461127848415
Median MAE: 266.7453797681334
Mean sMAPE: 0.08342404474803945
Median sMAPE: 0.05350304161549477


,res_without_adj,res_with_adj
0,540.603638,541.245103
1,334.970485,335.655023
2,421.787818,422.084611
3,263.022020,266.745380
4,0.083506,0.083424
5,0.054440,0.053503
6,500.538379,501.411746
7,322.454076,323.487057
8,408.311260,409.194444
9,265.227770,261.093929


In [ ]:
metrics_m3(num_series_m3_4, true_demand_lst_m3_4,
           pred_test_demand_lst_without_adj_m3_4, pred_test_demand_lst_with_adj_m3_4)

Mean RMSE: 259.95955937702337
Median RMSE: 119.54805736585274
Mean MAE: 205.59527699109387
Median MAE: 95.9089870661837
Mean sMAPE: 0.03290803017580682
Median sMAPE: 0.01569868304902074
Mean RMSE: 259.244766718338
Median RMSE: 121.25173789049323
Mean MAE: 204.35380400788338
Median MAE: 93.95835851347059
Mean sMAPE: 0.03282934605114459
Median sMAPE: 0.01569868304902074


,res_without_adj,res_with_adj
0,259.959559,259.244767
1,119.548057,121.251738
2,205.595277,204.353804
3,95.908987,93.958359
4,0.032908,0.032829
5,0.015699,0.015699
6,258.350555,256.403377
7,120.086866,117.835629
8,212.815446,210.506397
9,98.421847,97.463524


In [ ]:
metrics_m3(num_series_m3_5, true_demand_lst_m3_5,
           pred_test_demand_lst_without_adj_m3_5, pred_test_demand_lst_with_adj_m3_5)

Mean RMSE: 869.4094294306525
Median RMSE: 720.5373899257645
Mean MAE: 695.4555734056319
Median MAE: 583.559265816144
Mean sMAPE: 0.20550620161391844
Median sMAPE: 0.1805063528385018
Mean RMSE: 874.3556212299451
Median RMSE: 731.2553969463543
Mean MAE: 699.2515987330528
Median MAE: 588.0510805628298
Mean sMAPE: 0.2077494941318925
Median sMAPE: 0.18310946941254513


,res_without_adj,res_with_adj
0,869.409429,874.355621
1,720.537390,731.255397
2,695.455573,699.251599
3,583.559266,588.051081
4,0.205506,0.207749
5,0.180506,0.183109
6,874.541173,878.549493
7,750.999763,751.961282
8,727.456367,730.868579
9,618.828031,624.163338


In [ ]:
num_series_m3 = num_series_m3_1 + num_series_m3_2 + num_series_m3_3 + num_series_m3_4 + num_series_m3_5
true_demand_lst_m3 = true_demand_lst_m3_1 + true_demand_lst_m3_2 + true_demand_lst_m3_3 + true_demand_lst_m3_4 + true_demand_lst_m3_5
pred_test_demand_lst_without_adj_m3 = pred_test_demand_lst_without_adj_m3_1 + pred_test_demand_lst_without_adj_m3_2 + pred_test_demand_lst_without_adj_m3_3 + pred_test_demand_lst_without_adj_m3_4 + pred_test_demand_lst_without_adj_m3_5
pred_test_demand_lst_with_adj_m3 = pred_test_demand_lst_with_adj_m3_1 + pred_test_demand_lst_with_adj_m3_2 + pred_test_demand_lst_with_adj_m3_3 + pred_test_demand_lst_with_adj_m3_4 + pred_test_demand_lst_with_adj_m3_5

In [ ]:
metrics_m3(num_series_m3, true_demand_lst_m3,
           pred_test_demand_lst_without_adj_m3, pred_test_demand_lst_with_adj_m3)

Mean RMSE: 568.1394973786358
Median RMSE: 336.8229803979282
Mean MAE: 450.2313636388081
Median MAE: 266.8010175282418
Mean sMAPE: 0.10929029145370747
Median sMAPE: 0.054830705462964036
Mean RMSE: 565.9788915443769
Median RMSE: 337.8087607031114
Mean MAE: 448.49627975277036
Median MAE: 267.4240452802027
Mean sMAPE: 0.10975552394515359
Median sMAPE: 0.05443975721535998


,res_without_adj,res_with_adj
0,568.139497,565.978892
1,336.822980,337.808761
2,450.231364,448.496280
3,266.801018,267.424045
4,0.109290,0.109756
5,0.054831,0.054440
6,562.182610,558.385697
7,328.167800,325.942275
8,464.153270,460.970623
9,270.498211,268.921924


In [ ]:
# m3 lstm
def build_lstm_model(input_len, output_len):
  model = Sequential()
  model.add(LSTM(units = 16, return_sequences = False, input_shape=(input_len, 1)))
  model.add(Dropout(0.5))
  model.add(Dense(4))
  model.add(Dense(output_len))
  model.compile(loss = 'mean_squared_error', optimizer = 'adam')
  print(model.summary())
  return(model)

In [ ]:
num_series_m3_1, true_demand_lst_m3_1, pred_test_demand_lst_without_adj_m3_1, pred_test_demand_lst_with_adj_m3_1 = \
run_main_m3(m3_1_data, m3_1_info, input_len = 24, output_len = 1, seasonal_period = 12,
            build_model = build_lstm_model, batch_size = 32, epochs = 30, random_seed = 3407)

num_series: 111
total number of windows: 9028
start training


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 16)                  │           1,152 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 4)                   │              68 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │               5 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,225 (4.79 KB)

 Trainable params: 1,225 (4.79 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/30
283/283 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 1.6117
Epoch 2/30
283/283 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.7369
Epoch 3/30
283/283 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.6275
Epoch 4/30
283/283 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - loss: 0.5657
Epoch 5/30
283/283 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.5408
Epoch 6/30
283/283 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.5138
Epoch 7/30
283/283 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 0.5066
Epoch 8/30
283/283 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.5327
Epoch 9/30
283/283 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.5089
Epoch 10/30
283/283 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - loss: 0.5009
Epoch 11/30
283/283 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.4875
Epoch 12/30
283/283 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.4816
Epoch 13/30
283/283 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - loss: 0.4858
Epoch 14/30
283/283 ━━━━━━━━━━━━━━━━━━━━ 8s 27ms/step - loss: 0.5079
Epoch 15/30
283/283 ━━━━━━━━━━━━━━━━━━━━ 8s 1

100%|██████████| 85/85 [06:39<00:00,  4.70s/it]


train_rmse_with_adj: 942.4992207680765
start testing
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
metrics before adjusting
metrics after adjusting


In [ ]:
metrics_m3(num_series_m3_1, true_demand_lst_m3_1,
           pred_test_demand_lst_without_adj_m3_1, pred_test_demand_lst_with_adj_m3_1)

Mean RMSE: 304.95444162659265
Median RMSE: 54.117659097952895
Mean MAE: 221.80014254113394
Median MAE: 46.48894435112556
Mean sMAPE: 0.04094203377949507
Median sMAPE: 0.006763904825725358
Mean RMSE: 297.63171250048794
Median RMSE: 47.60004517182031
Mean MAE: 212.94838163956436
Median MAE: 40.03643143921343
Mean sMAPE: 0.039311104569396564
Median sMAPE: 0.006098980027476417


,res_without_adj,res_with_adj
0,304.954442,297.631713
1,54.117659,47.600045
2,221.800143,212.948382
3,46.488944,40.036431
4,0.040942,0.039311
5,0.006764,0.006099
6,306.751005,294.775433
7,49.435802,42.409502
8,230.997013,215.763028
9,40.814085,33.966013


In [ ]:
num_series_m3_2, true_demand_lst_m3_2, pred_test_demand_lst_without_adj_m3_2, pred_test_demand_lst_with_adj_m3_2 = \
run_main_m3(m3_2_data, m3_2_info, input_len = 24, output_len = 1, seasonal_period = 12,
            build_model = build_lstm_model, batch_size = 32, epochs = 30, random_seed = 3407)

num_series_m3_3, true_demand_lst_m3_3, pred_test_demand_lst_without_adj_m3_3, pred_test_demand_lst_with_adj_m3_3 = \
run_main_m3(m3_3_data, m3_3_info, input_len = 24, output_len = 1, seasonal_period = 12,
            build_model = build_lstm_model, batch_size = 32, epochs = 30, random_seed = 3407)

num_series_m3_4, true_demand_lst_m3_4, pred_test_demand_lst_without_adj_m3_4, pred_test_demand_lst_with_adj_m3_4 = \
run_main_m3(m3_4_data, m3_4_info, input_len = 24, output_len = 1, seasonal_period = 12,
            build_model = build_lstm_model, batch_size = 32, epochs = 30, random_seed = 3407)

num_series_m3_5, true_demand_lst_m3_5, pred_test_demand_lst_without_adj_m3_5, pred_test_demand_lst_with_adj_m3_5 = \
run_main_m3(m3_5_data, m3_5_info, input_len = 24, output_len = 1, seasonal_period = 12,
            build_model = build_lstm_model, batch_size = 32, epochs = 30, random_seed = 3407)

num_series: 145
total number of windows: 11948
start training


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 16)                  │           1,152 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 4)                   │              68 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │               5 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,225 (4.79 KB)

...

 Trainable params: 1,225 (4.79 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/30
751/751 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - loss: 1.1549
Epoch 2/30
751/751 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - loss: 1.0888
Epoch 3/30
751/751 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - loss: 1.0443
Epoch 4/30
751/751 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - loss: 1.0376
Epoch 5/30
751/751 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - loss: 1.0340
Epoch 6/30
751/751 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - loss: 1.0295
Epoch 7/30
751/751 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - loss: 1.0293
Epoch 8/30
751/751 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - loss: 1.0307
Epoch 9/30
751/751 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - loss: 1.0279
Epoch 10/30
751/751 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - loss: 1.0280
Epoch 11/30
751/751 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - loss: 1.0266
Epoch 12/30
751/751 ━━━━━━━━━━━━━━━━━━━━ 12s 11ms/step - loss: 1.0215
Epoch 13/30
751/751 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - loss: 1.0233
Epoch 14/30
751/751 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - loss: 1.0244
Epoch 15/30
751/751 ━━━━━━━━━━━━━━━━━━━

100%|██████████| 44/44 [01:29<00:00,  2.04s/it]


train_rmse_with_adj: 996.6617519046671
start testing
267/267 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
metrics before adjusting
metrics after adjusting


In [ ]:
metrics_m3(num_series_m3_2, true_demand_lst_m3_2,
           pred_test_demand_lst_without_adj_m3_2, pred_test_demand_lst_with_adj_m3_2)

Mean RMSE: 579.591233565423
Median RMSE: 206.88717586075234
Mean MAE: 473.9928412632314
Median MAE: 178.24391526432146
Mean sMAPE: 0.07525752572510827
Median sMAPE: 0.025181696605333193
Mean RMSE: 524.1810688920239
Median RMSE: 204.65037461260692
Mean MAE: 418.8686698108396
Median MAE: 168.93358451512086
Mean sMAPE: 0.06782806453681546
Median sMAPE: 0.024570786751736958


,res_without_adj,res_with_adj
0,579.591234,524.181069
1,206.887176,204.650375
2,473.992841,418.868670
3,178.243915,168.933585
4,0.075258,0.067828
5,0.025182,0.024571
6,568.443702,519.660039
7,206.106928,200.834286
8,476.317009,424.482567
9,176.287665,173.234661


In [ ]:
metrics_m3(num_series_m3_3, true_demand_lst_m3_3,
           pred_test_demand_lst_without_adj_m3_3, pred_test_demand_lst_with_adj_m3_3)

Mean RMSE: 605.4644447480059
Median RMSE: 388.15368095675603
Mean MAE: 470.6760159675485
Median MAE: 302.81613605459063
Mean sMAPE: 0.08943306227851938
Median sMAPE: 0.06393638197799142
Mean RMSE: 562.906967962871
Median RMSE: 366.02466157648166
Mean MAE: 441.1385117972028
Median MAE: 286.4704363265089
Mean sMAPE: 0.08630444833948381
Median sMAPE: 0.06089355787918019


,res_without_adj,res_with_adj
0,605.464445,562.906968
1,388.153681,366.024662
2,470.676016,441.138512
3,302.816136,286.470436
4,0.089433,0.086304
5,0.063936,0.060894
6,567.332309,522.693142
7,355.191236,343.300164
8,464.430953,430.454284
9,297.080580,286.890062


In [ ]:
metrics_m3(num_series_m3_4, true_demand_lst_m3_4,
           pred_test_demand_lst_without_adj_m3_4, pred_test_demand_lst_with_adj_m3_4)

Mean RMSE: 312.37269655020975
Median RMSE: 125.82710091972217
Mean MAE: 246.5758222496859
Median MAE: 96.94180871361394
Mean sMAPE: 0.03869835984308983
Median sMAPE: 0.015614044902881674
Mean RMSE: 283.603347201778
Median RMSE: 118.64883713316804
Mean MAE: 224.52518215669204
Median MAE: 94.4970925813085
Mean sMAPE: 0.035474646332222526
Median sMAPE: 0.015229340896567108


,res_without_adj,res_with_adj
0,312.372697,283.603347
1,125.827101,118.648837
2,246.575822,224.525182
3,96.941809,94.497093
4,0.038698,0.035475
5,0.015614,0.015229
6,313.760603,279.349178
7,115.696592,112.374767
8,257.222556,229.604533
9,91.637709,89.431961


In [ ]:
metrics_m3(num_series_m3_5, true_demand_lst_m3_5,
           pred_test_demand_lst_without_adj_m3_5, pred_test_demand_lst_with_adj_m3_5)

Mean RMSE: 831.8044985780713
Median RMSE: 706.5582326821495
Mean MAE: 667.1686255828598
Median MAE: 568.7874301564511
Mean sMAPE: 0.19821265368276111
Median sMAPE: 0.1734446455109609
Mean RMSE: 845.0194641053333
Median RMSE: 712.5470498377961
Mean MAE: 678.5583239296947
Median MAE: 576.3475400034229
Mean sMAPE: 0.20112771811030855
Median sMAPE: 0.17477775907455656


,res_without_adj,res_with_adj
0,831.804499,845.019464
1,706.558233,712.547050
2,667.168626,678.558324
3,568.787430,576.347540
4,0.198213,0.201128
5,0.173445,0.174778
6,831.651865,846.772633
7,707.779459,716.271237
8,693.136461,707.179713
9,592.005680,600.612011


In [ ]:
num_series_m3 = num_series_m3_1 + num_series_m3_2 + num_series_m3_3 + num_series_m3_4 + num_series_m3_5
true_demand_lst_m3 = true_demand_lst_m3_1 + true_demand_lst_m3_2 + true_demand_lst_m3_3 + true_demand_lst_m3_4 + true_demand_lst_m3_5
pred_test_demand_lst_without_adj_m3 = pred_test_demand_lst_without_adj_m3_1 + pred_test_demand_lst_without_adj_m3_2 + pred_test_demand_lst_without_adj_m3_3 + pred_test_demand_lst_without_adj_m3_4 + pred_test_demand_lst_without_adj_m3_5
pred_test_demand_lst_with_adj_m3 = pred_test_demand_lst_with_adj_m3_1 + pred_test_demand_lst_with_adj_m3_2 + pred_test_demand_lst_with_adj_m3_3 + pred_test_demand_lst_with_adj_m3_4 + pred_test_demand_lst_with_adj_m3_5

In [ ]:
metrics_m3(num_series_m3, true_demand_lst_m3,
           pred_test_demand_lst_without_adj_m3, pred_test_demand_lst_with_adj_m3)

Mean RMSE: 590.0082922115141
Median RMSE: 372.033394745431
Mean MAE: 467.8227850338468
Median MAE: 293.7022328792792
Mean sMAPE: 0.10999581097192072
Median sMAPE: 0.06348633329816272
Mean RMSE: 571.2774510823978
Median RMSE: 354.7712257118373
Mean MAE: 453.05377383749436
Median MAE: 282.8204087914637
Mean sMAPE: 0.10859514226044224
Median sMAPE: 0.06052549380600884


,res_without_adj,res_with_adj
0,590.008292,571.277451
1,372.033395,354.771226
2,467.822785,453.053774
3,293.702233,282.820409
4,0.109996,0.108595
5,0.063486,0.060525
6,579.984725,560.448735
7,358.744301,337.669300
8,478.653120,462.290114
9,297.640836,282.300820
